In [3]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [46]:
from streaming import LocalDataset
from streaming import MDSWriter
from datasets import Audio
from glob import glob
from tqdm import tqdm
import json
import mp

In [5]:
from collections import defaultdict

with open('prepared-indonesian-ms.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        break

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [41]:
!split -l 30000 -d --additional-suffix=.splitted prepared-indonesian-original.jsonl prepared-indonesian-original.jsonl

In [39]:
def loop(files):
    files, _ = files
    indons = []
    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                t = tokenizer.decode(l['predict_id'], decode_with_timestamps = True).split('<|endoftext|>')[0]
                t = t + '<|endoftext|>'
                indons.append([l['filename'], t])
    return indons

In [45]:
indons = mp.multiprocessing(sorted(glob('prepared-indonesian-original.jsonl*splitted')), loop, cores = 10)

30000it [00:36, 831.02it/s] 
30000it [00:40, 743.06it/s] 
30000it [00:40, 740.64it/s]
30000it [00:40, 735.88it/s]
30000it [00:41, 723.39it/s] 
30000it [00:43, 687.84it/s] 
30000it [00:43, 682.40it/s]
30000it [00:44, 672.34it/s]
30000it [00:45, 661.19it/s] 
30000it [00:46, 639.39it/s]
13734it [00:18, 750.70it/s] 
30000it [00:36, 826.45it/s] 
30000it [00:36, 831.80it/s] 
30000it [00:36, 812.30it/s]
30000it [00:37, 803.21it/s]


In [51]:
filenames = defaultdict(dict)

with open('prepared-indonesian-ms.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        filenames[l['audio_filename']]['ms'] = l['new_text']
len(filenames)

320620

In [52]:
with open('prepared-indonesian-en.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        filenames[l['audio_filename']]['en'] = l['new_text']
        
len(filenames)

320934

In [56]:
for i in range(len(indons)):
    if indons[i][0] not in filenames:
        continue
    filenames[indons[i][0]]['id'] = indons[i][1]
    
len(filenames)

320934

In [57]:
columns = {
    'audio_filename': 'str',
    'new_text': 'str',
    'new_text_en': 'str',
    'new_text_ms': 'str'
}
hashes = 'sha1', 'xxh64'

In [68]:
!rm -rf mosaic-indon

In [69]:
with MDSWriter(out='mosaic-indon', columns=columns, compression=None, hashes=hashes) as out:
    for k, v in tqdm(filenames.items()):
        d = {
            'audio_filename': k,
            'new_text': v['id'],
            'new_text_en': v.get('en', ''),
            'new_text_ms': v.get('ms', ''),
        }
        out.write(d)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 320934/320934 [00:02<00:00, 155071.04it/s]


In [70]:
dataset = LocalDataset('mosaic-indon')

In [71]:
len(dataset)

320934

In [72]:
dataset[0]

{'audio_filename': 'split-indonesian/1-160-118560000.mp3',
 'new_text': '<|startoftranscript|><|id|><|transcribe|><|0.00|> Ada sekitar 20 ribu orang total yang terdeteksi ya di sini, di area konvoy, di Nasdem Tower tadi, dan juga di KPU total untuk mengikuti jalannya konvoy ini.<|14.96|><|16.42|> Dan jumlah tersebut adalah gabungan dari kader partai anggota koalisi dengan relawan.<|24.28|><|24.82|><|endoftext|>',
 'new_text_en': '<|startoftranscript|><|en|><|transcribe|><|0.00|> There are approximately 20 thousand people in total who are detected here, in the convoy area, at Nasdem Tower, and also at the KPU in total to follow the progress of this convoy.<|14.96|><|16.42|> And this number is a combination of coalition and volunteer party cadres.<|24.28|><|endoftext|>',
 'new_text_ms': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Terdapat kira-kira 20 ribu orang kesemuanya yang dikesan di sini, di kawasan konvoi, di Menara Nasdem, dan juga di KPU kesemuanya untuk mengikuti perkemb